# Preprocesamiento para Experimentos de Actividad Espintanea
 
Este Cuaderno es muy similar al Preprocesamiento para Evocados, pero sin promediar sobre múltiples estímulos. En lugar de eso se recomienda una inspección visual para obtener un intervalo interesante del Archivo Experimental.

In [2]:
include("SeparaActividadySaturados01.jl")
# en este modulo se encuentran las rutinas para encontrar los estimulos ("trancazos")
# y los canales saturados
using SeparaActividadySaturados01
using JLD

LoadError: LoadError: LoadError: [91mArgumentError: Module Libdl not found in current path.
Run `Pkg.add("Libdl")` to install the Libdl package.[39m
while loading /home/karel/.julia/v0.6/HDF5/src/HDF5.jl, in expression starting on line 10
while loading /home/karel/RafaelGutierrez/Preprocesamiento/SeparaActividadySaturados01.jl, in expression starting on line 3

In [4]:
Pkg.add("Libdl")

LoadError: [91m[91munknown package Libdl[39m
macro expansion at ./pkg/entry.jl:53 [inlined]
(::Base.Pkg.Entry.##1#3{String,Base.Pkg.Types.VersionSet})() at ./task.jl:335[39m